# TESTING / EXPERIMENTING WITH ML MODELS (notebook to be deleted)


---

In [2]:
# Data Management
import numpy as np
import pandas as pd

# Modeling
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Visualization
import geopandas as gpd
import folium
import folium.plugins as plugins
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots

# Utils
import math
from datetime import datetime, timedelta
import ipywidgets as widgets
import random
import warnings
from IPython.display import display, HTML
from ipywidgets import interact, interact_manual
from time import sleep
%matplotlib inline
sns.set_style('white')
sns.set_color_codes()
plt.style.use('default')  # sub 'dark_background'
warnings.filterwarnings("ignore")

In [8]:
new_patients = pd.read_csv('./uc_new_patients.csv', index_col='pt_id')
new_patients.head()

,pt_name,pt_dob,pt_age,visit_location,visit_reason,visit_code,visit_date,visit_day,checkin_time,checkout_time,rolling_ct,rolling_code,assigned_num_techs,needed_num_techs
pt_id,,,,,,,,,,,,,,
1026443,James Fisher,1997-09-03,24,denver,covid-test,4,2022-05-01,Sunday,09:15:12,09:39:12,1,0.0,4,1
1026456,Chad Taylor,2009-06-23,12,denver,cold/flu/fever,4,2022-05-01,Sunday,09:43:36,10:41:36,1,0.0,4,1
1026438,Daisy Lawson,2005-06-17,16,denver,diarrhea,5,2022-05-01,Sunday,10:04:01,11:10:01,2,4.0,4,1
1026442,Cindy Mckinney,1992-03-11,30,denver,cold/flu/fever,4,2022-05-01,Sunday,10:07:59,11:12:59,3,4.5,4,1
1026452,Gene Webster,1969-07-18,52,denver,vaccination,3,2022-05-01,Sunday,10:32:39,10:54:15,4,4.3,4,2


In [9]:
past_patients = pd.read_csv('./uc_past_patients.csv', index_col='pt_id')
past_patients.head()

,pt_name,pt_dob,pt_age,visit_location,visit_reason,visit_code,visit_date,visit_day,checkin_time,checkout_time,rolling_ct,rolling_code,assigned_num_techs,needed_num_techs
pt_id,,,,,,,,,,,,,,
1000019,Jeffrey Landry,1996-06-25,25,denver,chest-pain,5,2021-05-01,Saturday,08:40:27,09:58:27,1,0.0,5,1
1000023,Christopher Wright,1974-04-26,48,denver,cough,4,2021-05-01,Saturday,08:49:20,09:39:20,2,5.0,5,1
1000022,Lori Walters,2018-12-20,3,denver,stomach-pain,5,2021-05-01,Saturday,09:29:43,11:00:43,2,5.0,5,1
1000011,Meghan Richards,1974-05-27,47,denver,rash/allergy,5,2021-05-01,Saturday,09:50:21,10:44:21,3,5.0,5,1
1000003,Erica Thompson,1999-06-24,22,denver,covid-test,4,2021-05-01,Saturday,09:55:42,10:25:42,4,5.0,5,2


In [13]:
new_patients['weekend'] = new_patients['visit_day'].copy().apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
past_patients['weekend'] = past_patients['visit_day'].copy().apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)


pt_id
1000019    1
1000023    1
1000022    1
1000011    1
1000003    1
          ..
5023061    1
5023069    1
5023075    1
5023062    1
5023070    1
Name: weekend, Length: 109174, dtype: int64

In [16]:
new_patients[1000019]['checkin_time']

KeyError: 1000019

if there is one tech available, use past data to try and predict whether or not we should keep this tech because we'll get busy or could we trasfer them.

In [15]:
past_patients.groupby(['visit_location','weekend'])['rolling_ct']

visit_location  weekend
denver          0          7.323444
                1          6.827364
edgewater       0          4.805536
                1          4.689442
lakewood        0          6.487402
                1          5.974275
rino            0          5.370688
                1          5.189064
wheatridge      0          5.977370
                1          5.789541
Name: rolling_ct, dtype: float64